In [1]:
import datasets # type: ignore

input_dataset = 'lmsys/chatbot_arena_conversations'
dataset = datasets.load_dataset(input_dataset)['train']
n_rows = len(dataset)
print(str(n_rows))

33000


In [2]:
from typing import Generator

def fixed_chunker(dataset: datasets.Dataset, 
                  chunk_size: int) -> Generator:
    for i in range(0, len(dataset), chunk_size):
        yield dataset.select(range(i, min(i + chunk_size, len(dataset))))

def add_order_column(dataset: datasets.Dataset) -> datasets.Dataset:
    return dataset.map(
        lambda example, idx: {'order': idx}, 
        with_indices=True)
    
def add_question_column(dataset: datasets.Dataset) -> datasets.Dataset:
    return dataset.map(
        lambda example: {'question': example['conversation_a'][0]['content']})

# Test:
for big_chunk in fixed_chunker(dataset, 10000):
    print(f"Chunk len: {len(big_chunk)}")

for big_chunk in fixed_chunker(dataset, 100):
    big_chunk = add_order_column(big_chunk)
    print(f"First Order: {big_chunk[0]['order']}")

    big_chunk = add_question_column(big_chunk)
    print(f"First Question: {big_chunk[0]['question']}")

    # Sort the chunk.
    big_chunk = big_chunk.sort('question')
    print(f"First Question after sort: {big_chunk[0]['question']}")

    # Unsort the chunk.
    big_chunk = big_chunk.sort('order')
    print(f"First Question after unsort: {big_chunk[0]['question']}")

    break

Chunk len: 10000
Chunk len: 10000
Chunk len: 10000
Chunk len: 3000
First Order: 0
First Question: What is the difference between OpenCL and CUDA?
First Question after sort: 3,14 + 9855 + 0,000001 = ?
First Question after unsort: What is the difference between OpenCL and CUDA?


In [3]:
import torch
import utils.llm_utils as llm

model_name = 'microsoft/Phi-3-mini-4k-instruct'
tokenizer, model = llm.load_tokenizer_and_model(model_name)

In [ ]:
def token_len(tokenizer, text: str) -> torch.Tensor:
    message = [{"role": "user", "content": text}] 
    tokens = tokenizer.apply_chat_template([message], add_generation_prompt=True)[0]
    return len(tokens)

def add_token_len_column(tokenizer, dataset: datasets.Dataset) -> datasets.Dataset:
    return dataset.map(
        lambda example: {'token_len': token_len(tokenizer, example['question'])})

def token_chunker(dataset: datasets.Dataset, 
                  chunk_tokens: int,
                  generated_tokens:int) -> Generator:
    ''' Yield chunks of examples that when prompted, tokenized, padded and generated are 
    smaller than chunk_tokens size. 
    Since examples are sorted descending by token_len, all examples in a chunk will be
    padded to the length of the first example in the chunk. '''
    start = 0
    while start < len(dataset):
        example_len = dataset[start]['token_len'] + generated_tokens
        n_chunk = min(chunk_tokens//example_len, len(dataset)-start)
        yield dataset.select(range(start, start+n_chunk))
        start += n_chunk

def chunk_call_llm(chunk: datasets.Dataset, tokenizer, model,
                   gen_tokens: int) -> None:
    messages = [[{"role": "user", "content": question}] 
                for question in chunk['question']]
    answers = llm.batch_call_llm(tokenizer, model, messages, gen_tokens)
    return chunk.map(
        lambda example, idx: {'answer': answers[idx]},
        with_indices=True)

def process_variable_chunks(dataset: datasets.Dataset, 
                            tokenizer,
                            model,
                            big_chunk_size: int,
                            small_chunk_tokens: int,
                            max_gen_tokens: int) -> Generator:

    for big_chunk in fixed_chunker(dataset, 100):
        big_chunk = big_chunk.select_columns(['question_id', 'conversation_a'])
        big_chunk = add_question_column(big_chunk)
        big_chunk = add_token_len_column(tokenizer, big_chunk)
        big_chunk = add_order_column(big_chunk)
        big_chunk = big_chunk.sort('token_len', reverse=True)
        # print(f"First Question tokenization len: {big_chunk[0]['token_len']}")

        big_chunk = datasets.concatenate_datasets(
            [chunk_call_llm(small_chunk, tokenizer, model, max_gen_tokens) 
            for small_chunk in token_chunker(big_chunk, 1500, max_gen_tokens)] )

        big_chunk = big_chunk.sort('order')
        # print(f"First Question tokenization len: {big_chunk[0]['token_len']}")
        # print(f"First answer: {big_chunk[0]['answer']}")

        big_chunk = big_chunk.select_columns(['question_id', 'question', 'answer'])
        yield big_chunk

# Test:
for chunk in process_variable_chunks(dataset, tokenizer, model, 100, 1500, 50):
    print(f"Chunk len: {len(chunk)}")
    print(f"First answer: {chunk[0]['answer']}")
    break

In [ ]:
out_jsonl = "1_dataset_creator/phi3_arena_brief_dataset.jsonl"
with open(out_jsonl, 'w') as f: pass

for chunk in process_variable_chunks(dataset, tokenizer, model, 100, 1500, 50):
    with open(out_jsonl, 'ab') as f:
        chunk.to_json(f, lines=True)
    break